In [1]:
import pandas as pd
import folium
from folium.plugins import MeasureControl
import branca.colormap as cm

# Load your station and rainfall data
epmaps_stations = pd.read_csv('EPMAPS_stations.csv')
fonag_stations = pd.read_csv('FONAG_stations.csv')
rainfall_data = pd.read_csv('all_stations_daily_rainfall.csv')

# Merge the EPMAPS and FONAG stations
merged_stations = pd.concat([epmaps_stations, fonag_stations], axis=0)

# Merge the rainfall data with the station metadata
rainfall_with_stations = pd.merge(rainfall_data, merged_stations, left_on='station_id', right_on='id_estacion', how='left')

# Prepare the data with coordinates, station names, and codes
rainfall_with_station_info = rainfall_with_stations[['fecha', 'PRECIPITACION SUM', 'station_id', 'punto_obs', 'codigo', 'latitud', 'longitud']]

# Calculate the date of the previous day for the title
from datetime import datetime, timedelta
yesterday = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')

# Set up the color scale (red for high, blue for low rainfall)
min_rainfall = rainfall_with_station_info['PRECIPITACION SUM'].min()
max_rainfall = rainfall_with_station_info['PRECIPITACION SUM'].max()
colormap = cm.linear.RdYlBu_09.scale(min_rainfall, max_rainfall)

# Create the map centered on the general area of the stations
m = folium.Map(location=[-0.2, -78.5], zoom_start=10, tiles='OpenStreetMap')

# Add a scale bar (MeasureControl includes distance scale)
m.add_child(MeasureControl())

# Add rainfall data as markers with radius proportional to rainfall value and solid color
for _, row in rainfall_with_station_info.iterrows():
    folium.CircleMarker(
        location=[row['latitud'], row['longitud']],
        radius=max(3, row['PRECIPITACION SUM'] * 0.5),  # Scale the marker size
        popup=f"Station: {row['punto_obs']}<br>Rainfall: {row['PRECIPITACION SUM']} mm",
        color=colormap(row['PRECIPITACION SUM']),  # Marker border color
        fill=True,
        fill_color=colormap(row['PRECIPITACION SUM']),  # Marker fill color
        fill_opacity=1,  # Solid fill
        opacity=1  # Solid border
    ).add_to(m)

# Add the color scale bar to the map
colormap.add_to(m)

# Add a title using folium's HTML feature
title_html = f'''
    <h3 align="center" style="font-size:20px"><b>Precipitación Diaria del {yesterday}</b></h3>
    '''
m.get_root().html.add_child(folium.Element(title_html))

# Save the map as an HTML file
m.save('rainfall_map_with_solid_points.html')

# To open the map, you can simply use any web browser to open the generated 'rainfall_map_with_solid_points.html' file.


BORDER LINES

In [6]:
import pandas as pd
import folium
from folium.plugins import MeasureControl
import geopandas as gpd
import branca.colormap as cm

# Load your station and rainfall data
epmaps_stations = pd.read_csv('EPMAPS_stations.csv')
fonag_stations = pd.read_csv('FONAG_stations.csv')
rainfall_data = pd.read_csv('all_stations_daily_rainfall.csv')

# Merge the EPMAPS and FONAG stations
merged_stations = pd.concat([epmaps_stations, fonag_stations], axis=0)

# Merge the rainfall data with the station metadata
rainfall_with_stations = pd.merge(rainfall_data, merged_stations, left_on='station_id', right_on='id_estacion', how='left')

# Prepare the data with coordinates, station names, and codes
rainfall_with_station_info = rainfall_with_stations[['fecha', 'PRECIPITACION SUM', 'station_id', 'punto_obs', 'codigo', 'latitud', 'longitud']]

# Calculate the date of the previous day for the title
from datetime import datetime, timedelta
yesterday = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')

# Set up the color scale (red for high, blue for low rainfall)
min_rainfall = rainfall_with_station_info['PRECIPITACION SUM'].min()
max_rainfall = rainfall_with_station_info['PRECIPITACION SUM'].max()
colormap = cm.linear.RdYlBu_09.scale(min_rainfall, max_rainfall)

# Create the map centered on the general area of the stations
m = folium.Map(location=[-0.2, -78.5], zoom_start=10, tiles='CartoDB positron')

# Load the GeoPackage with polygons and add boundaries to the map (added first)
gdf = gpd.read_file('Area.gpkg')

# Add the polygon boundaries (outline only) to the map
for _, poly in gdf.iterrows():
    folium.GeoJson(
        poly['geometry'],
        style_function=lambda feature: {
            'color': 'black',  # Boundary color
            'weight': 2,       # Boundary thickness
            'fillOpacity': 0,  # No fill, only boundary
        }
    ).add_to(m)

# Add a scale bar (MeasureControl includes distance scale)
m.add_child(MeasureControl())

# Add rainfall data as markers, with radius proportional to rainfall value and solid color (added last for layering on top)
for _, row in rainfall_with_station_info.iterrows():
    folium.CircleMarker(
        location=[row['latitud'], row['longitud']],
        radius=max(3, row['PRECIPITACION SUM'] * 0.5),  # Scale the marker size
        popup=f"Station: {row['punto_obs']}<br>Rainfall: {row['PRECIPITACION SUM']} mm",
        color=colormap(row['PRECIPITACION SUM']),  # Marker border color
        fill=True,
        fill_color=colormap(row['PRECIPITACION SUM']),  # Marker fill color
        fill_opacity=1,  # Solid fill
        opacity=1  # Solid border
    ).add_to(m)

# Add the color scale bar to the map
colormap.add_to(m)

# Add a title using folium's HTML feature
title_html = f'''
    <h3 align="center" style="font-size:20px"><b>Precipitación Diaria del {yesterday} - Red de estaciones EPMAPS- FONAG</b></h3>
    '''

logo_html_footer = '''
     <div style="position: fixed; bottom: 10px; left: 50px; width: 100%; text-align: center; z-index: 9999;">
         <img src="Fonag-Color.png" style="width:150px; height:auto; display: inline-block;">
         <img src="Logo EPMAPS 15 5 23 (Interno).jpg" style="width:150px; height:auto; display: inline-block;">
         <img src="Logo EPMAPS y Alcaldía Quito Renace 15 5 23.jpg" style="width:150px; height:auto; display: inline-block;">
         <img src="inamhi-logo-LETRA-AZUL.png" style="width:150px; height:auto; display: inline-block;">
     </div>
'''

# Add the logos to the map as HTML
m.get_root().html.add_child(folium.Element(logo_html_footer))
m.get_root().html.add_child(folium.Element(title_html))
# Save the map as an HTML file
m.save('rainfall_map_with_gpkg_borders_and_points.html')

# To open the map, you can simply use any web browser to open the generated 'rainfall_map_with_gpkg_borders_and_points.html' file.
